<div style="font-family: 'Source Code Pro'; font-size: 24px;">

Al tener multiples procesos corriendo aparece la necesidad de sincronizar y comunicar esos procesos e hilos.

Supongamos que tenemos un recurso que no puede ser utilizado por dos personas a la vez. No puedo imprimir media pagina de una hoja y media pagina de otra por ejemplo. Que hago si tengo multiples procesos que quierenn usar ese dispositivo? tengo que serializar. Normalmente uno trata de serializar esos procesos para que se hagan de a uno a la vez. Un codigo que podriamos usar para llegar a cargo esta tarea. Tenemos una variable que se llama flag, 0 si el recurso esta libre 1 si esta usado. Levanto en el registro AX la variable flag, testeo, si no es 0 salto a ocupado, si es 0 incremento el RAX, y lo muevo a la variable flag, a partir de ahi empiezo a utilizar el recurso. En algun momento terminare de utilizarlo, borro el registro AX y grabo el 0 en flag. Si hay una sola tarea no habria problema con ese codigo.

Ahora que pasa si tenemos dos threads ejecutando el codigo anterior. Se fija si el flag esta en 0 para saber si esta libre, si no esta en 0 se altera y si esta en 0 lo pone en 1, lo usa y luego lo pone en 0 devuelta. Se ejecutan algunos pedazos del codigo de los threads que estan ejecutables. El problema es que habra un uso simultaneo de recursos, que es justamente lo que queriamos evitar. 


## **Task Switch**

Un task switch o cambio de tarea es el proceso mediante el cual el SO pausa la ejecucion de una tarea (o hilo) y transfiere el control a otra. Esto es fundamental para la multitarea, permitiendo que multiples tareas comparten el tiempo de CPU de manera eficiente.

### **Caracteristicas clave**

- Context switching: Implica guardar el esado (contexto) de la tarea actual y restuarar el contexto de la tarea siguiente.

- Interrupciones: Los tasks swithces suelen ocurrir en respuesta a interrupciones, como las de temporzador o de E/S.

- Final de instruccion: El cambio de tarea generalmente ocurre al final de una instruccion para mantener la coherencia y evitar inconsistencias en el estado de la CPU.

### **Operaciones Atomicos y la instruccion XCHG**

Las operaciones atomicas son aquellas que se ejecutan indivisiblemente, sin posibilidad de interrupciones que puedan causar inconsistencias en entornos concurrentes. En arquitecturas x86, la instruccion XCHG (exchange) es un ejemplo de operacion atomica.

### **Funcionamiento de XCHG**

- Intercambio de valores: La instruccion XCHG intercambia el contenido de dos operandos. Por ejemplo, XCHG AX, flag intercambiara el valor del registro AX con el de flag.

- Atomicidad Garantizada: Esta operacion se realiza de manera atomica, lo que significa que no puede ser interrumpida. Es util para implementar mecanismos de sincronizacion como locks (bloqueos).

### **Ejemplo detallado**

```assembly

; Inicialmente, flag = 0 (recurso libre)
MOV AX, 1          ; Cargar 1 en AX
XCHG AX, flag      ; Intercambiar AX con flag de manera atómica
; Ahora:
; - AX contiene el valor anterior de flag
; - flag contiene 1 (indicando que el recurso está ocupado)

```

- Interpretacion:

    - Si AX originalmente era 0, significa que el recurso estaba libre y ahora esta bloqueado (flag = 1).

    - Si AX era 1, el recurso ya estaba bloqueado por otra tarea.

Esta operacion permite verificar y establecer el estado del flag en una sola instruccion, evitando condiciones de carrera en entornos de un solo nucleo.

### **Limitaciones en sistemas multicore**

En sistemas con multiples nucleos, cada nucleo puede ejecutar hilos de manera concurrente. Aunque las operaciones atomicas como XCHG funcionan bien en entornos de un solo nucelo, surgen desafios cuando multiples nucleos intentan ejecutar estas operaciones simultaneamente.

**Escenario problematico:**

Imagina dos hilos, thread1 y thread2, ejecutándose en dos núcleos distintos y ambos intentando realizar XCHG AX, flag al mismo tiempo cuando flag inicialmente es 0.

1. Ejecución Simultánea:

- CPU1 (Core1): Ejecuta XCHG AX, flag, leyendo flag = 0 y estableciendo flag = 1.

- CPU2 (Core2): Al mismo tiempo, ejecuta XCHG AX, flag, también leyendo flag = 0 y estableciendo flag = 1.

2. Resultado Inesperado:

- Ambos hilos creen haber adquirido el bloqueo, ya que ambos leyeron flag = 0.

- Ahora, ambos hilos piensan que tienen acceso exclusivo al recurso, lo que lleva a una condición de carrera.

**¿Por Qué Ocurre Esto?**

Aunque XCHG es atómica a nivel de un solo núcleo, no proporciona sincronización entre múltiples núcleos. Cada núcleo puede tener su propia caché de la memoria, y sin una coherencia de caché adecuada, las operaciones simultáneas pueden pasar por alto las actualizaciones realizadas por otros núcleos.

Coherencia de Caché:

- Problema: Cada núcleo puede mantener una copia local de la memoria caché. Sin mecanismos adecuados, un núcleo puede no ver las actualizaciones realizadas por otro núcleo de manera inmediata.

- Solución: Utilizar barreras de memoria (memory barriers) o instrucciones específicas que aseguren la coherencia entre cachés.

### **Soluciones para la Sincronización en Sistemas Multicore**

Para manejar la sincronización de manera efectiva en entornos multicore, es necesario utilizar mecanismos que aseguren la visibilidad y la ordenación correcta de las operaciones entre los núcleos.

**a. Instrucciones de Sincronización Avanzadas:**

- LOCK Prefix: En arquitecturas x86, el prefijo LOCK puede usarse con ciertas instrucciones (incluyendo XCHG) para asegurar que la operación se realice de manera exclusiva en todos los núcleos, implementando una barrera de memoria implícita.

```assembly

LOCK XCHG AX, flag

```
- Efecto: Garantiza que la operación XCHG sea atómica a nivel de sistema, evitando que otros núcleos realicen operaciones concurrentes en flag durante el intercambio.

**b. Operaciones Atómicas de Alto Nivel:**

- Primitives de Sincronización: Lenguajes y bibliotecas modernos proporcionan primitivas de sincronización como mutexes, semáforos y variables atómicas que manejan la complejidad de la sincronización en entornos multicore.

- Instrucciones de Comparación y Cambio (Compare-and-Swap): Otra técnica común para implementar bloqueos sin necesidad de prefijos de bloqueo.

**c. Diseño de Algoritmos Sincronizados:**

- Evitar Condiciones de Carrera: Diseñar algoritmos que minimicen o eliminen la necesidad de compartir recursos entre hilos.

- Bloqueos de Nivel Superior: Utilizar bloqueos que gestionen la exclusividad de acceso a recursos compartidos de manera eficiente.

### **Explicacion de dani**

El task switch es siempre algo que se da al final de una instruccion, no a la mitad de una. Veamos el siguiente codigo. Cargo AX con 1, lo intercambio en una unica instruccion con el flag, en AX quedo el valor del flag y en flag quedo un 1, ahora yo puedo comparar ver si estaba ocupado o no. La idea es en vez de levantar el valor del flag, compararlo, hacer el jump condicional y despues ponerle un 1 al flag, lo que hace es pone un 1 en el registro y lo intercambia con el flag, ahora va a mirar el registro que en realidad tiene el contenido del flag, si el registro estaba en 0 quiere decir que yo lo pase de 0 a 1 y que por ende lo tengo yo, si el registro estaba en 1 ya estaba en 1 cuando lo intercambie por ende lo tiene otro. Porque funciona esto? la funcion exchange es una funcion unica, y por ende cuando el SO hace un task switch lo hace al final de cada instruccion. Si no se ejecuto el exchange porque llego un task swithc queda el RAX en 1 y le toca al otro porque me toma el recurso. Si ya se ejecuto el exchange cuando me interrumpen, el recurso ya es mio y cuando me devuelvan el control lo usare y cuando termine lo liberare, pero no habra un uso simultaneo del recurso. Este tipo de instrucciones se llama Read-Modify-Write, son en un solo paso.

Esto funciona siempre?

Ahora tenemos dos cores, en un core ejecuta el thread1 y en otro core el thread2. Por ejemplo el thread1 va a ejecutar el exchange, el CPU1 le dije al manejador de memoria (MMU) lee la posicion 3, este lee de memoria, levanta el valor y se lo devuelve a la CPU1, ahora simultaneamente el otro core va a ejecutar el exchange tambien, el MMU lo lee de memoria y se lo devuelve a la CPU2, el problema es que a ambos le devolvio el valor 0. Los dos RAX quedaron en 0 y otra vez tenemos el problema que se pisan entre si, el problema es que cuando tengo multiples procesadores las instrucciones se ejecutan en paralelo entonces el read-modify-write ya no funciona. Estos errores de programacion son dificiles de encontrar y es mucho mas razonable pensar 10 veces antes de hacerlo porque lleva mucho tiempo. 

## Lock

### **¿Qué es el Prefijo LOCK?**

El prefijo LOCK es una directiva utilizada en la arquitectura x86 que se aplica a ciertas instrucciones de ensamblador para asegurar que se ejecuten de manera atómica en entornos multiprocesador. Cuando se utiliza LOCK, se garantiza que la operación modificadora de memoria sea exclusiva y que ningún otro procesador pueda acceder a la misma región de memoria hasta que la operación se complete.

### **Características Clave:**

- Atomicidad Garantizada: LOCK asegura que la instrucción modificadora de memoria se ejecute completamente sin interrupciones.

- Exclusividad del Bus: Al aplicar LOCK, se bloquea el bus de memoria, evitando que otros núcleos accedan o modifiquen la misma dirección de memoria simultáneamente.

- Sincronización entre Núcleos: Facilita la coherencia de caché y la 

- sincronización entre múltiples núcleos al coordinar el acceso a recursos compartidos.

### **Funcionamiento del Prefijo LOCK**

**a. Modo de Operación:**

Cuando se antepone LOCK a una instrucción, se activa una serie de acciones a nivel de hardware para asegurar la exclusividad de la operación:

- Adquisición del Bus: El procesador que ejecuta la instrucción LOCK adquiere el control del bus de memoria, impidiendo que otros procesadores accedan a la misma dirección de memoria.

- Coherencia de Caché: Se asegura que todas las cachés de los demás núcleos estén actualizadas con respecto a la dirección de memoria involucrada, evitando inconsistencias.

- Ejecución de la Instrucción: La instrucción modificadora de memoria (como ADD, SUB, XCHG, etc.) se ejecuta de manera atómica.

- Liberación del Bus: Una vez completada la instrucción, el bus se libera, permitiendo que otros procesadores realicen operaciones de memoria.

**b. Instrucciones Comunes con LOCK:**

El prefijo LOCK se puede aplicar a varias instrucciones que modifican la memoria, incluyendo pero no limitadas a:

- LOCK ADD

- LOCK SUB

- LOCK INC

- LOCK DEC

- LOCK XCHG

- LOCK CMPXCHG

### **Importancia de LOCK en Entornos Multicore**

En sistemas con múltiples núcleos, varias tareas pueden intentar acceder y modificar la misma región de memoria simultáneamente. Sin mecanismos de sincronización adecuados, esto puede llevar a:

- Condiciones de Carrera: Donde el resultado de operaciones depende del orden de ejecución de los hilos.

- Inconsistencias de Datos: Donde diferentes hilos ven estados diferentes de los mismos datos.

El prefijo LOCK es esencial para evitar estos problemas, ya que asegura que las operaciones críticas se realicen de manera exclusiva y consistente.

### **Ejemplo detallado de uso de LOCK**

Consideremos nuevamente la implementacion de un mecanismo de bloqueo utilizando la instruccion XCHG. Veams como se aplica LOCK en este contexto:


Sin LOCK:

```assembly

; Intento de adquirir un bloqueo sin LOCK
MOV AX, 1          ; Cargar 1 en AX
XCHG AX, flag      ; Intercambiar AX con flag
; Verificar el valor de AX para determinar si el bloqueo fue exitoso

```

En un entorno multicore, dos hilos podrian ejecutar esta secuencia simultanemaente y ambos podrian leer flag = 0, creyendo haber adquirido el bloqueo.

Con LOCK:

```assembly

; Intento de adquirir un bloqueo con LOCK
MOV AX, 1          ; Cargar 1 en AX
LOCK XCHG AX, flag ; Intercambiar AX con flag de manera atómica
; Verificar el valor de AX para determinar si el bloqueo fue exitoso

```
Flujo con LOCK:

- Adquisición del Bus: El núcleo que ejecuta LOCK XCHG AX, flag adquiere el bus de memoria.

- Intercambio Atómico: La instrucción XCHG se ejecuta de manera atómica, intercambiando AX con flag.

- Liberación del Bus: Una vez completada la operación, el bus se libera para otros núcleos.

Resultado:

- Si flag era 0, AX ahora contiene 0 y flag contiene 1, indicando que el bloqueo fue adquirido con éxito.

- Si flag era 1, AX contiene 1, indicando que otro hilo ya tenía el bloqueo.

La exclusividad garantizada por LOCK evita que múltiples hilos adquieran el bloqueo simultáneamente.

### **LOCK y Coherencia de Caché**

En sistemas multicore, cada núcleo puede tener su propia caché. Sin LOCK, es posible que un núcleo lea una versión obsoleta de una variable desde su caché, ignorando las actualizaciones realizadas por otros núcleos. Al usar LOCK, se fuerza una sincronización de caché, asegurando que todas las operaciones de memoria sean visibles para todos los núcleos de manera inmediata.

### **Implementación a Nivel de Hardware**

a. Bloqueo del Bus:

El prefijo LOCK utiliza mecanismos de hardware para bloquear el bus de memoria durante la ejecución de la instrucción. Esto asegura que ningún otro núcleo pueda acceder a la memoria compartida hasta que la operación se complete.

b. Barreras de Memoria Implícitas:

Además de bloquear el bus, LOCK actúa como una barrera de memoria, garantizando que todas las operaciones de memoria anteriores se completen antes de la instrucción LOCK y que ninguna operación posterior comience antes de que la instrucción LOCK finalice.

### **Instrucciones XCHG y LOCK**

Es interesante notar que en la arquitectura x86, la instrucción XCHG ya es intrínsecamente atómica cuando uno de los operandos es un registro. Esto se debe a que XCHG con un registro implícitamente utiliza LOCK. Por lo tanto, no es necesario anteponer explícitamente LOCK a XCHG en estos casos.

### **Alternativas y Prácticas Recomendadas**

Aunque LOCK es una herramienta poderosa, su uso debe manejarse con cuidado debido a:

- Impacto en el Rendimiento: El uso excesivo de LOCK puede causar contención en el bus de memoria y reducir el rendimiento en sistemas con muchos núcleos.

- Complejidad del Código: Implementar correctamente mecanismos de sincronización a bajo nivel puede ser complejo y propenso a errores.

**a. Uso de Primitivas de Alto Nivel:**

En lugar de manipular directamente instrucciones con LOCK, es recomendable utilizar primitivas de sincronización proporcionadas por el lenguaje de programación o las bibliotecas estándar, como mutexes, semáforos, y variables atómicas. Estas abstraen la complejidad de la sincronización y están optimizadas para el rendimiento.

**b. Minimizar el Uso de Recursos Compartidos:**

Diseñar algoritmos que reduzcan al mínimo la necesidad de compartir recursos entre hilos puede disminuir la dependencia de mecanismos de sincronización y mejorar el rendimiento.

**c. Evitar el Uso Innecesario de LOCK:**

Usar LOCK solo cuando sea estrictamente necesario. Cada operación bloqueante puede impactar el rendimiento, por lo que es importante evaluar si una operación atómica es realmente requerida.


### **Explicacion de dani**

Podemos pensar en la siguiente alternativa, el prefijo LOCK. Este prefijo bloquea el bus. Cuando se va a ejecutar la ejecucion de la instruccion el bus va a quedar para mia, no pasara mas esto de que el MMU me atiende a mi simultaneamente que antiende a otros cores, de ultima si la MMU esta ocupado porque otro tiene el lock a mi me frena y la instruccion no se ejecuta. Este LOCK lo que garantiza es que el cambio es atomico, todo se produce en un unico modulo que no puede ser roto en subpedazos. Este prefijo tiene esa funcion y es lo unico que da la solucion en un ambiente multicore. Es tan comun olvidarse el LOCK que ya hoy en dia el mismo procesador agrega el LOCK como prefijo de exchange.

## Spinlocks

Un spinlock es un tipo de mecanismo de sincronizacion que se utiliza para controlar el acceso exclusivo a un recurso compartido en entornos concurrentes, como en SO con multiples hilos o nucleos de CPU. A diferencia de otros mecanismos de bloqueo que ponen a los hilos en espera (bloqueo de hilos), los spinlocks mantienen al hilo en un bucle de espera activo ("girando") hasta que el bloqueo este disponible.

### **Características Clave:**

- Espera Activa (Busy Waiting): El hilo que intenta adquirir el spinlock permanece en un ciclo de espera, verificando repetidamente si el bloqueo está disponible.

- Baja Latencia: Son eficientes para secciones críticas que se ejecutan rápidamente, ya que evitan el overhead de poner y sacar hilos del estado de espera.

- Uso en Contextos de Kernel: Comúnmente utilizados en el núcleo del sistema operativo donde la espera activa es aceptable y los hilos no pueden ser puestos en espera.

### **Ventajas y Desventajas:**

**Ventajas:**

- Baja Latencia: Ideal para operaciones que requieren acceso rápido y exclusivo a recursos compartidos.

- Simplicidad: Implementación sencilla en comparación con otros mecanismos de sincronización más complejos.

**Desventajas:**

- Consumo de CPU: La espera activa consume ciclos de CPU innecesariamente, lo que puede afectar el rendimiento si el bloqueo se mantiene por mucho tiempo.

- Ineficiente para Esperas Prolongadas: No es adecuado cuando se espera que el bloqueo esté ocupado durante un período significativo.

### **Spinlocks en Windows**

En Windows, los spinlocks son principalmente utilizados dentro del núcleo (kernel) y están diseñados para funcionar en entornos multiprocesador. Windows proporciona funciones y estructuras específicas para manejar spinlocks de manera eficiente.

**a. Definición y Inicialización**

En Windows, un spinlock se representa mediante la estructura KSPIN_LOCK.

**b. Adquirir y Liberar un Spinlock**

Windows proporciona funciones como KeAcquireSpinLock y KeReleaseSpinLock para manejar spinlocks.

**c. Uso de Interlocked Exchange**

Además de las funciones de alto nivel, Windows ofrece funciones de intercambio atómico (interlocked) que permiten implementar spinlocks de manera manual si es necesario.

**d. Consideraciones en Windows**

Irqls (Interrupt Request Levels): En el núcleo de Windows, la adquisición de spinlocks a menudo involucra cambiar el nivel de interrupciones para evitar que las interrupciones interfieran con la sección crítica.
Uso en el Kernel: Los spinlocks en Windows están destinados principalmente para uso dentro del núcleo y no deben ser utilizados en aplicaciones de espacio de usuario.

### **Spinlocks en Linux**

En Linux, los spinlocks son una herramienta fundamental para la sincronización dentro del núcleo del sistema operativo. Proporcionan mecanismos eficientes para proteger secciones críticas en entornos multiprocesador.

**a. Definición y Inicialización**

En Linux, los spinlocks se representan mediante la estructura spinlock_t. Para inicializar un spinlock, se puede usar la macro SPIN_LOCK_UNLOCKED o la función spin_lock_init.

**b. Adquirir y Liberar un Spinlock**

Linux proporciona varias funciones para manejar spinlocks, incluyendo spin_lock, spin_unlock, spin_trylock, entre otras.

**c. Diferencias Clave en la Implementación de Linux**

Interruptiones Locales: Al adquirir un spinlock en Linux, a menudo se deshabilitan las interrupciones locales para prevenir que el hilo actual sea interrumpido mientras mantiene el spinlock, evitando así condiciones de carrera.

### **¿Cuándo Utilizar Spinlocks?**

Los spinlocks son especialmente útiles en los siguientes escenarios:

- Secciones Críticas Breves: Cuando la sección crítica se ejecuta rápidamente, de modo que la espera activa no consume una cantidad significativa de CPU.

- Entornos de Kernel: Donde poner hilos en espera no es práctico o posible.

- Sin Preemption: En sistemas donde los hilos no son preemptibles mientras mantienen el spinlock.

Ejemplos de Uso:

- Protección de Estructuras de Datos Compartidas: Como listas enlazadas o tablas hash en el núcleo.

- Control de Acceso a Recursos de Hardware: Donde múltiples hilos pueden intentar acceder a registros de dispositivos simultáneamente.

### **Consideraciones de Rendimiento y Eficiencia**

Aunque los spinlocks pueden ser muy eficientes para secciones críticas breves, su uso indebido puede llevar a problemas de rendimiento:

- Contención de Spinlocks: Si múltiples hilos intentan adquirir el mismo spinlock simultáneamente, puede provocar una alta contención y desperdicio de ciclos de CPU.

- Esperas Prolongadas: Si la sección crítica tarda mucho en ejecutarse, los hilos en espera activa consumen recursos sin progreso real.

- Impacto en la Escalabilidad: En sistemas con muchos núcleos, el uso excesivo de spinlocks puede escalar mal y afectar negativamente el rendimiento general.



### Explicacion de dani

Que distintos mecanismos nos ofrece el SO para sincronizar las tareas sin necesidad de escribir codigo assembler o instrucciones raras. El SO nos da varios mecanismos para sincronizar tareas y tener acceso exlusivo a un recurso. El priimero es el spinlock, es como una variable que utilizara el SO y que en principio tiene como valores 0 o 1. El SO tiene que darme funciones para que pueda manejar esa variable, en windows defino una var con un spinlock. Cuando le quiero poner el 0 al principio para inicializar y decir que el recurso no esta utilizado le pongo un 0 y listo. Cuando la quiero intentar setear existe el interlockexchange que hace el lock exchange adentro, esta funcion garantiza que ningun otro thread te va a poder tocar. Cuando lo quiero liberar ejecuto el interlockexchange() en 0, el valor que devuelve lo tiro, esta es la forma de hacerlo en windows. La realidad es que es muy raro usar spinlocks. En Linux es distinto, el spinlock es una estructura, lo inicializo pasandole un puntero y poniendole un 0, tengo dos funciones, una que es el trylock y una funcion lock.

## Semaforo

### **¿Qué es un Semáforo?**

Un semaforo es un mecanismo de sincronización utilizado para controlar el acceso de múltiples procesos o hilos a recursos compartidos de manera coordinada. Los semáforos ayudan a prevenir condiciones de carrera y garantizan que los recursos se utilicen de manera eficiente y segura.

### **Características Clave:**

- Contador Interno: Un semáforo mantiene un contador que representa el número de permisos disponibles.

- Operaciones Básicas:

    - Wait (P): Decrementa el contador. Si el contador es mayor que cero, el hilo puede continuar; de lo contrario, el hilo se bloquea hasta que un permiso esté disponible.

    - Signal (V): Incrementa el contador y, si hay hilos bloqueados esperando, uno de ellos se desbloquea.

- Tipos de Semáforos:

    - Semáforo Binario: Similar a un mutex, solo puede tomar los valores 0 o 1.

    - Semáforo Contador: Puede tomar múltiples valores positivos, permitiendo controlar múltiples accesos simultáneos a un recurso.

### **Tipos de Semáforos**

a. Semáforo Binario

- Descripción: Solo puede tener dos valores: 0 (no disponible) y 1 (disponible).

- Uso Común: Controlar el acceso exclusivo a un recurso, similar a un mutex.

- Ventaja: Simple y eficiente para la sincronización de acceso exclusivo.

b. Semáforo Contador

- Descripción: Puede tener cualquier valor entero no negativo, permitiendo múltiples accesos simultáneos.

- Uso Común: Gestionar recursos limitados, como conexiones de red o espacios en buffer.

- Ventaja: Permite un control granular sobre la cantidad de hilos que pueden acceder a un recurso simultáneamente.

### **Ventajas de los Semáforos sobre los Spinlocks**

a. Control de Cantidad de Accesos

- Semáforos: Permiten controlar el número de hilos que acceden a un recurso simultáneamente mediante un contador.

- Spinlocks: Solo permiten el acceso exclusivo, sin capacidad para manejar múltiples accesos.

b. Eficiencia en Espera

- Semáforos: Utilizan espera pasiva, donde los hilos bloqueados no consumen ciclos de CPU mientras esperan.

- Spinlocks: Utilizan espera activa (busy waiting), lo que puede resultar en un alto consumo de CPU si la espera es prolongada.

c. Flexibilidad en la Sincronización

- Semáforos: Son más versátiles y pueden adaptarse a diferentes escenarios de sincronización.

- Spinlocks: Son más adecuados para secciones críticas muy breves donde la espera activa es aceptable.

### **Implementación y Uso de Semáforos en Windows y Linux**

a. Semáforos en Windows

En Windows, los semáforos se gestionan a través de la API de sincronización proporcionada por el sistema operativo. Se utilizan principalmente en aplicaciones de espacio de usuario para sincronizar hilos.

Creación y Inicialización de un Semáforo

Para crear un semáforo en Windows, se utiliza la función CreateSemaphore o CreateSemaphoreEx.

Adquirir (Wait) y Liberar (Signal) un Semáforo

- Adquirir (Wait): Se utiliza WaitForSingleObject o WaitForMultipleObjects para esperar a que el semáforo esté disponible.

- Liberar (Signal): Se utiliza ReleaseSemaphore para incrementar el contador del semáforo y desbloquear hilos en espera.

Cerrar el Handle del Semáforo

Una vez que se ha terminado de utilizar el semáforo, es importante cerrar su handle para liberar recursos.

b. Semáforos en Linux

En Linux, los semáforos pueden ser utilizados tanto en el espacio de usuario como en el kernel. En el espacio de usuario, se gestionan a través de la biblioteca POSIX (pthread). Existen dos tipos principales:

- Semáforos Anónimos: No tienen nombre y son utilizados entre hilos dentro del mismo proceso.

- Semáforos con Nombre: Pueden ser compartidos entre diferentes procesos.

### **Aplicaciones y Casos de Uso de Semáforos**

a. Thread Pools

Un thread pool es un grupo de hilos que se mantienen disponibles para ejecutar tareas a medida que se van solicitando. Los semáforos son esenciales en la gestión de thread pools para controlar el número de tareas que se pueden ejecutar simultáneamente y sincronizar el acceso a las colas de tareas.

Funcionamiento:

- Contador del Semáforo: Representa el número de tareas disponibles para ser ejecutadas.

- Hilos del Pool: Cada hilo espera a que el semáforo indique que hay una tarea disponible.

- Asignación de Tareas: Cuando se añade una tarea a la cola, se incrementa el semáforo (Signal), lo que desbloquea un hilo para que la ejecute.

Ventajas:

- Eficiencia: Evita la creación y destrucción constante de hilos.

- Control: Limita el número de tareas concurrentes, evitando la sobrecarga del sistema.

b. Gestión de Recursos Limitados

Cuando se dispone de un número limitado de recursos (como conexiones de base de datos, accesos a archivos, etc.), los semáforos permiten controlar cuántos hilos pueden acceder a dichos recursos simultáneamente.

Ejemplo:

- Conexiones a Base de Datos: Si solo hay 10 conexiones disponibles, se puede inicializar un semáforo con un valor de 10. Cada vez que un hilo quiera usar una conexión, decrementar el semáforo. Al liberar una conexión, incrementar el semáforo.

c. Sincronización entre Hilos

Los semáforos también se utilizan para sincronizar la ejecución de hilos, asegurando que ciertos hilos esperen a que otros completen tareas específicas antes de continuar.

Ejemplo:

- Productor-Consumidor: En el problema clásico de productor-consumidor, se utilizan semáforos para controlar el número de elementos disponibles y el espacio libre en el buffer.

### **Buenas Prácticas al Usar Semáforos**

Para utilizar semáforos de manera efectiva y evitar problemas de sincronización, es importante seguir ciertas buenas prácticas:

a. Inicializar Correctamente el Semáforo

- Valor Inicial Adecuado: Asegúrate de que el valor inicial del semáforo refleja correctamente la cantidad de recursos disponibles.

- Consistencia: Evita cambios no sincronizados en el contador del semáforo.

b. Evitar el Uso de Semáforos para Sincronización Compleja

- Simplicidad: Utiliza semáforos para casos sencillos de sincronización. Para sincronizaciones más complejas, considera otras primitivas como mutexes, barreras o variables de condición.

c. Manejar Correctamente la Liberación de Semáforos

- Liberar Siempre: Asegúrate de que cada adquisición de semáforo (wait) tenga una correspondiente liberación (signal), incluso en casos de error o excepciones.

- Evitar Liberaciones Duplicadas: No liberes un semáforo más veces de lo necesario, ya que esto puede llevar a inconsistencias en la sincronización.

d. Prevenir Condiciones de Carrera

- Acceso Atómico: Asegúrate de que las operaciones de adquisición y liberación del semáforo sean atómicas y estén correctamente sincronizadas.

- Bloqueo Ordenado: Si necesitas adquirir múltiples semáforos, hazlo en un orden consistente para prevenir deadlocks.

e. Documentar el Uso de Semáforos

- Claridad en el Código: Documenta claramente por qué y cómo se están utilizando los semáforos en el código para facilitar el mantenimiento y la comprensión.

- Comentarios Informativos: Incluye comentarios que expliquen la lógica de sincronización y la finalidad de cada semáforo.

### **Consideraciones de Rendimiento y Eficiencia**

a. Contención de Semáforos

- Problema: Si muchos hilos intentan adquirir un semáforo al mismo tiempo, puede generar una alta contención, lo que afecta el rendimiento.

- Solución: Diseña el sistema para minimizar la contención, posiblemente aumentando el número de recursos o optimizando la lógica de sincronización.

b. Uso de Semáforos en Sistemas de Alta Carga

- Eficiencia: En sistemas con alta concurrencia, el uso eficiente de semáforos es crucial para mantener un rendimiento óptimo.

- Optimización: Evalúa el impacto de los semáforos en el rendimiento y considera alternativas si es necesario.

c. Evitar Deadlocks

- Definición: Un deadlock ocurre cuando dos o más hilos esperan indefinidamente por recursos que nunca se liberan.

- Prevención: Implementa estrategias para evitar la adquisición circular de recursos y utiliza tiempos de espera (timeouts) si es posible.

### Explicacion de dani

Otra alternativa es el semaforo. Son parecidos pero tienen 0 (inactivo) y tienen multiples valores positivos. Es muy util porque cuando esperamos un semaforo, si esta en 0 se queda esperando, cuando algun otro thread incrementa el semaforo, arbitrariamente se elije algun lo de los otros que esten esperando y se le dice que siga y luego obviamente se baja a 0 devuelta. Cuando alguien incrementa nuevamente el semaforo, el SO agarra otra tarea arbitrariamente y le dice que siga y lo vuelve a llevar a 0. Si alguien lo incrementara a 3, el SO agarra 3 de los que esten esperando arbitrariamente y los haace seguir. La ventaja que tiene el semaforo frente al spinlock es que uno puede controlar cantidades, cada vez que se incremento el semaforo un thread que estaba esperando en ese semaforo es liberado. Como se usan? en windows hay que tener un HANDLE, el primero tiene que crear y definir la estructura del semaforo y los otros abrir la estructura del semaforo, conectarse al semaforo. Cuando uno termina hay que hacer el CloseHandle(). En linux es una estructura y la dinamica es similar. Muchas veces los lenguajes de programacion tienen sus propias bilbiotecas que exponen estas funciones con los mismos nombres. El uso mas importante de los semaforos son los thread pools.

## Mutex

### **¿Qué es un Mutex?**

Un mutex es un mecanismo de sincronización utilizado para controlar el acceso exclusivo a un recurso compartido entre múltiples hilos (threads) dentro de un mismo proceso. Su propósito principal es asegurar que solo un hilo pueda ejecutar una sección crítica de código a la vez, evitando condiciones de carrera y garantizando la integridad de los datos compartidos.

### **Características Clave:**

- Exclusión Mutua: Garantiza que solo un hilo pueda poseer el mutex en un momento dado.

- Propiedad: El hilo que adquiere el mutex es responsable de liberarlo. Esto ayuda a mantener un control claro sobre quién tiene acceso al recurso.

- Bloqueo y Desbloqueo: Los mutexes proporcionan operaciones para bloquear (adquirir) y desbloquear (liberar) el acceso al recurso.

- Evita Espera Activa: A diferencia de los spinlocks, los mutexes utilizan espera pasiva, lo que significa que los hilos que intentan adquirir un mutex bloqueado serán puestos en espera sin consumir ciclos de CPU.

### **Funcionamiento de un Mutex**

a. Adquisición del Mutex (Lock)

Cuando un hilo intenta adquirir un mutex:

- Estado Disponible: Si el mutex no está poseído por ningún otro hilo, el hilo lo adquiere inmediatamente y continúa su ejecución.

- Estado No Disponible: Si el mutex ya está poseído, el hilo que intenta adquirirlo será bloqueado y puesto en una cola de espera hasta que el mutex sea liberado.

b. Liberación del Mutex (Unlock)

El hilo que posee el mutex debe liberarlo una vez que haya terminado de acceder al recurso compartido. Al liberar el mutex, si hay hilos en la cola de espera, el sistema operativo seleccionará uno de ellos (generalmente de manera arbitraria o siguiendo un orden específico) para que adquiera el mutex y continúe su ejecución.

### **Ventajas de los Mutexes sobre Spinlocks y Semáforos**

a. Exclusión Mutua con Propiedad Clara

- Mutexes: Tienen una propiedad clara donde el hilo que adquiere el mutex es el único responsable de liberarlo, lo que facilita la gestión y previene errores como liberar un mutex que no se posee.

- Spinlocks: No tienen una propiedad intrínseca, lo que puede llevar a que cualquier hilo libere el spinlock, potencialmente causando inconsistencias.

- Semáforos: No gestionan la propiedad, ya que cualquier hilo puede incrementar (signal) el semáforo, lo que puede ser útil pero también puede introducir errores si no se maneja correctamente.

b. Espera Pasiva y Eficiencia de CPU

- Mutexes y Semáforos: Utilizan espera pasiva, donde los hilos bloqueados no consumen ciclos de CPU, lo que los hace más eficientes para secciones críticas de duración variable o prolongada.

- Spinlocks: Utilizan espera activa (busy waiting), consumiendo ciclos de CPU mientras esperan, lo que puede ser ineficiente si la espera es larga.

c. Flexibilidad en la Sincronización

- Mutexes: Ideales para sincronizar el acceso exclusivo a recursos compartidos en aplicaciones de espacio de usuario.

- Semáforos: Más versátiles para controlar el acceso a múltiples instancias de un recurso.

- Spinlocks: Adecuados principalmente para secciones críticas muy breves, especialmente en contextos de kernel donde la espera activa es aceptable.

### **Buenas Prácticas al Usar Mutexes**

Para utilizar mutexes de manera efectiva y evitar problemas como condiciones de carrera, deadlocks o comportamientos inesperados, es importante seguir ciertas buenas prácticas:

a. Minimizar la Duración de la Sección Crítica

- Descripción: Mantén la cantidad de código dentro de la sección crítica lo más breve posible.

- Razón: Reduce el tiempo durante el cual otros hilos deben esperar para adquirir el mutex, mejorando la concurrencia y el rendimiento general.

b. Evitar Deadlocks

- Descripción: Un deadlock ocurre cuando dos o más hilos esperan indefinidamente por recursos que nunca se liberan.

- Prevención:

    - Orden Consistente: Si múltiples mutexes deben ser adquiridos por varios hilos, establece un orden consistente para adquirirlos.

    - Evitar Bloqueos Anidados: Minimiza la necesidad de adquirir múltiples mutexes simultáneamente.

    - Tiempos de Espera: Implementa tiempos de espera (timeouts) al adquirir mutexes para detectar y manejar deadlocks potenciales.

c. Uso Adecuado de la Propiedad del Mutex

- Descripción: Asegúrate de que el hilo que adquiere el mutex sea el único responsable de liberarlo.

- Razón: Previene errores donde un hilo no poseedor intenta liberar el mutex, lo que puede causar comportamientos indefinidos.

d. Inicializar y Destruir Correctamente los Mutexes

- Descripción: Siempre inicializa los mutexes antes de usarlos y destrúyelos cuando ya no sean necesarios.

- Razón: Evita fugas de recursos y garantiza que los mutexes funcionen correctamente durante toda la vida útil de la aplicación.

e. Manejar Excepciones y Errores Apropiadamente

- Descripción: Asegúrate de liberar los mutexes incluso si ocurre una excepción o error dentro de la sección crítica.

- Método: Utiliza bloques try-finally (en Java), bloques with (en Python) o estructuras similares en otros lenguajes para garantizar la liberación del mutex.

f. Documentar el Uso de Mutexes

- Descripción: Mantén una documentación clara sobre qué recursos están protegidos por qué mutexes y cómo se adquieren y liberan.

- Razón: Facilita el mantenimiento del código y ayuda a otros desarrolladores a comprender la lógica de sincronización.

### Proteccion de recursos compartidos:

- Un Solo Recurso Compartido: Si todos los hilos acceden a un único recurso compartido (por ejemplo, una variable global, una estructura de datos, etc.), entonces un solo mutex suele ser suficiente para garantizar la exclusión mutua durante el acceso a ese recurso.

- Múltiples Recursos Compartidos: Si tienes múltiples recursos compartidos que pueden ser accedidos de manera independiente, es más eficiente utilizar un mutex por cada recurso. Esto reduce la contención, ya que los hilos que acceden a diferentes recursos no estarán bloqueados entre sí.

### Explicacion de dani

Aparte de los spinlocks y los semaforos (semaforos serian un caso general de los spinlocks) tenemos otra estructura que son los Mutex. Los mutex permiten que un bloque de codigo no pueda ser ejecutado simultaenamente por dos threads. En definitiva es una estructura en memoria, hay que decirle al SO que la inicialice. Y podemos esperarlo, pero lo importante de esto es que yo vengo ejecutando, pongo el wait for single object y yo se que cuando salga de esa instruccion lo tengo yo, cuando hago release mutex al final lo libere. Hay que tener cuidado con uno programa esto, normalmente las tareas requieren datos y devuelven datos, de alguna forma deberia cargar los datos en una estructura, pasarle el puntero al thread y cuando termina devolver. De alguna forma esto involucra poner los datos en una lista encadenada.

Todos estos mecanismsos podrian combinarse.

El objetivo de los spinlocks, mutex y semaforos es coordinar los threads dentro de un mismo proceso.

## Pipes

### **¿Qué es un Pipe?**

Un pipe es un mecanismo de comunicación entre procesos (IPC, por sus siglas en inglés) que permite que un proceso envíe datos a otro proceso de manera unidireccional. Esencialmente, actúa como una tubería física a través de la cual los datos fluyen desde un extremo (escritor) al otro extremo (lector) en el orden en que se envían.

### **Características Clave:**

- Unidireccionalidad: Los pipes tradicionales son unidireccionales, lo que significa que los datos fluyen en una sola dirección: desde el proceso escritor al proceso lector.

- Orden de FIFO: Los datos se leen en el mismo orden en que se escribieron (FIFO: First In, First Out).

- Dos Extremos: Cada pipe tiene dos extremos:

    - Extremo de Escritura: Donde el proceso escribe los datos.

    - Extremo de Lectura: Donde el proceso lee los datos.

### **Funcionamiento de los Pipes**

a. Creación de un Pipe:

Cuando se crea un pipe, se generan dos descriptores de archivo (file descriptors) o handles:

- Descriptor de Escritura: Permite al proceso escribir datos en el pipe.

- Descriptor de Lectura: Permite al proceso leer datos desde el pipe.

Estos descriptores son utilizados por los procesos para enviar y recibir datos a través del pipe.

b. Comunicación a Través del Pipe:

- Proceso Escritor:

    - Abre el extremo de escritura del pipe.

    - Envía datos escribiendo en el pipe.

    - Los datos se almacenan en un búfer interno del sistema operativo.

- Proceso Lector:

    - Abre el extremo de lectura del pipe.

    - Lee los datos del pipe, que fluyen desde el extremo de escritura.

    - Los datos se leen en el orden en que se escribieron.

c. Cierre de los Extremos del Pipe:

- Proceso Escritor: Después de escribir los datos, cierra el extremo de escritura para indicar que no enviará más datos.

- Proceso Lector: Al cerrar el extremo de lectura, finaliza la comunicación. 

5. Limitaciones y Consideraciones de Uso

a. Comunicación entre Más de Dos Procesos:

- Descripción: Técnicamente, un pipe está diseñado para conectar dos procesos: uno que escribe y otro que lee.

- Posible pero No Recomendado: Aunque es posible conectar más de dos procesos a través de un pipe, generalmente no es práctico porque cada dato leído del pipe será consumido por un único proceso. Esto puede llevar a que otros procesos no reciban los datos, a menos que se implementen mecanismos adicionales para distribuir los datos.

- Alternativas: Para comunicaciones más complejas entre múltiples procesos, se recomiendan otros mecanismos de IPC como sockets, colas de mensajes o memoria compartida.

b. Persistencia de los Pipes:

- Pipes Anónimos: No persisten más allá de la vida de los procesos que los utilizan.

- Pipes Nombrados (FIFOs): Persisten en el sistema de archivos hasta que se eliminan explícitamente con comandos como rm en Linux.

c. Tamaño del Búfer del Pipe:

- Descripción: Los pipes tienen un tamaño de búfer limitado, lo que significa que si el proceso escritor envía datos más rápido de lo que el lector puede consumir, el escritor puede bloquearse hasta que haya espacio disponible en el búfer.

- Solución: Diseñar la aplicación para manejar adecuadamente los flujos de datos y evitar bloqueos innecesarios.

d. Seguridad y Permisos:

- Pipes Nombrados: En Linux, los permisos del archivo FIFO determinan qué procesos pueden leer o escribir en el pipe. Es crucial configurar correctamente los permisos para evitar accesos no autorizados.

### **Ventajas de los Pipes**

- Simplicidad: Los pipes son fáciles de usar y entender, especialmente para comunicaciones básicas entre procesos.

- Eficiencia: Ofrecen una comunicación eficiente sin la sobrecarga de otros mecanismos de IPC más complejos.

- Integración con la Línea de Comandos: Facilitan la construcción de pipelines en la shell para combinar comandos de manera poderosa y flexible.

### **Desventajas de los Pipes**

- Unidireccionalidad: Los pipes tradicionales son unidireccionales, lo que limita su uso a comunicaciones en una sola dirección.

- Limitación a Dos Procesos: Diseñados principalmente para la comunicación entre dos procesos, dificultando su uso para múltiples procesos.

- Falta de Persistencia (en Pipes Anónimos): No son adecuados para comunicaciones persistentes entre procesos no relacionados.

### **Alternativas a los Pipes**

Dependiendo de las necesidades de la aplicación, pueden considerarse otros mecanismos de IPC:

- Sockets: Permiten comunicaciones bidireccionales y pueden operar en la misma máquina o a través de una red.

- Memoria Compartida: Ofrece una forma rápida de compartir datos entre procesos, pero requiere mecanismos de sincronización adicionales.

- Colas de Mensajes: Permiten el envío de mensajes estructurados entre procesos, con colas gestionadas por el sistema operativo.

- Sockets de Dominio (Unix Domain Sockets): Proporcionan comunicación bidireccional eficiente entre procesos en la misma máquina.

### **Combinación de Mecanismos de IPC**

Los diferentes mecanismos de IPC pueden combinarse para satisfacer necesidades más complejas de comunicación y sincronización entre procesos Por ejemplo:

- Pipes y Semáforos: Usar pipes para la transmisión de datos y semáforos para sincronizar el acceso a esos pipes.

- Pipes y Mutexes: Proteger secciones críticas donde múltiples procesos escriben o leen desde un pipe utilizando mutexes.

- Memoria Compartida y Pipes: Utilizar memoria compartida para compartir grandes cantidades de datos y pipes para enviar señales o notificaciones sobre cambios en la memoria.

### Explicacion de dani

Mecanismo de comunicacion entre proceso. En el pipe uno puede poner cosas y le van a salir del otro lado en el orden en que se pusieron. Es importante entender que cuando crea un pipe crea dos extremos, el extremo de escritura y el extremo de lectura. Cuando la creo a mi me devuelve dos descriptores que permiten escribir o leer del pipe. En windows tenes createnamedpipe que permite crear un pipe, en linux se llama mkfifo, para abrirla en windows uso createfile y en linux hago open. Se envian mensajes y se reciben mensajes. El pipe es una estructura del SO que permite conectar distintos procesos. Tecnicamente el pipe es algo de dos extremos, por ende esta pensado para conectar dos procesos. Yo podria conectar un proceso y del otro lado poner dos procesos? no es muy practico, no dara error, pero no es practico porque el mensaje se pierde cuando es leido por uno de los procesos. La realidad es que se puede poner dos procesos en una punta pero lo normal es que hayan dos procesos uno en cada punta, uno escribe otro lee y listo. Cuando un proceso arranca en linux tiene conectado tres archivos stdin: recibe datos del teclado, stdout: pantalla, y el stderr: donde se guardan los errores. Cuando uno ejecuta ls | grep "a.txt" busca econtrar el texto "a.txt". En definitiva lo que pretenedemos con esto es que se ejecute ls, que el ls genere el listado pero que se lo pase al grep para que analice linea por linea y se fije si esta "a.txt". El primer programa el ls va a tener el stdin en el teclado, va a tener el stdout en una pata de un pipe que va a crear y el stderr va a valer la pantalla. Cuando ejeceute el ls si el ls detecta un error y lo imprime el stderr va a salir en la pantalla. Pero sino, todo lo que imprima en el stdout (listado de archivos del directorio) no van a ir a la pantalla, van a entrar en una pata del pipe. Y cual van a ser los descrpitores del segundo programa (grep), el grep va a tener el stdin en la pata de salida del pipe, va a tener el stdout en la pantalla y el stderr en la pantalla tambien. Entonces el grep lo que va a procesar es lo que viene del stdin pero el stdin tiene el stdout del otro.

## Signals

Las señales son mecanismos de comunicación asíncrona utilizados por el sistema operativo para notificar a los procesos sobre eventos específicos. Funcionan como interrupciones que pueden ser enviadas por el sistema operativo o por otros procesos, permitiendo que un proceso reaccione a ciertas condiciones o eventos sin necesidad de una supervisión constante.

### **Características Clave:**

- Asincronía: Las señales pueden ser enviadas en cualquier momento, independientemente de lo que el proceso esté ejecutando.

- Notificación de Eventos: Permiten notificar eventos como interrupciones del usuario, errores de ejecución, finalización de procesos hijos, entre otros.

- Interrupción de la Ejecución: Cuando una señal es recibida, puede interrumpir la ejecución normal del proceso para ejecutar una rutina de manejo de señales

### **Uso de Señales para Control de Procesos y Eventos**

a. Control de Terminación de Procesos:

Las señales permiten controlar cómo y cuándo terminan los procesos, proporcionando flexibilidad para manejar eventos inesperados o solicitudes de terminación.

b. Sincronización y Coordinación entre Procesos:

Las señales pueden utilizarse para coordinar la ejecución entre procesos, notificando sobre la finalización de tareas o la disponibilidad de recursos.

c. Implementación de Servicios y Daemons:

En sistemas Unix/Linux, los servicios (daemons) utilizan señales para gestionar su ciclo de vida, reinicios, y otras operaciones administrativas.


## Explicacion de dani

Cuando uno hace un programa, independiente del punto de entrada que empiece a ejecutar el programa, el SO tiene una funcion que uno tiene que escribir si o si, y que recibe notificaciones del SO. Por ejemplo, el caso de control c control break, etc. Cuando en un proceso ponemos control-c el proceso se corta, porque se corta? el SO llama a una funcion que llama a exitprocess, podria modificar esta funcion y cuando lo notifica si es un control c y no matarlo. En el caso de wiindows las notificaciones son bastantes limitadas: control-c, control-brake, LogOff Y Shutdown. En el caso de linux son mas de 30, pero lo mas importante es que no solamente las envia el SO. Cualquier tarea con el debido privilegio puede enviar seniales a otro prooceso para notificarlo. La senial SIGUSR1 es un caso tipico que es interpretada distinto por distintos programas dado que es de propositos generales. Bajo Linux las seniales pueden ser enviadas, inclusive, dessde la consola.

## Resumen de las ultimas dos clases:

La diferencia entre los threads y los procesos es que los procesos son una entidad mas compleja entre los cuales se proteje memoria, se los considera entidades independientes dentro de la maquina. Los threads en cambio tambien representaban un hilo de ejecucion con su propio stack y su propio juego de registros, pero entre los threads del mismo proceso no habia proteccion de memoria o de archivos. Cuando el SO quiere pasar de un thread a otro la cantidad de cosas que tiene que modfiicar en el procesador para hacer ese cambio es menor, mientras que cambie el stack y los registros perfectamente termino con lo que tenia que hacer. Cuando uno conmuta de un proceso a otro todo lo que es proteccion tambien tiene que ser reinicializado para indicar que este proceso no puede acceder a los del anterior. Hace falta involucrar al hardware en este proceso de proteccion de memoria. 

